<br>Transform Storm Tracker L0 data to L1 csv format file (batch mode).<br>
Hungjui Yu<br>
20210715<br>

In [21]:
%reset

import time
import pandas as pd
import datetime as dt
import pytz
# import metpy.calc as mpcalc
from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [37]:
# # Set ST node number:
# ST_no = 2968

# # Set ST launch time (UTC):
# launch_time_from_log = '20210503184852'

# # Set ST files path:
# ST_file_path = './'

# Set ST_info file path:
# ST_info_file = './log_online.xlsx'
ST_info_file = './log_online.csv'

# Set ST L0 files path:
ST_L0_file_path = '/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L0'


In [23]:
# Set output path:

output_path = '/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv'

# Set final date of launches:

final_date = dt.datetime(2021,6,25)


In [24]:
def read_in_ST_info(log_file):
    
    # ST_log = pd.read_excel(log_file)
    # ST_log = pd.read_excel(log_file, engine='openpyxl')
    ST_log = pd.read_csv(log_file)
    
    return ST_log

# ST_info = read_in_ST_info(ST_info_file)
# print(type(ST_info))
# ST_info.head()

In [49]:
def load_st_file(ST_no, launch_time_from_log, ST_file_path):    
    
    # Specified timezones:
    # pytz.all_timezones
    tz_utc = pytz.timezone('UTC')
    tz_fc = pytz.timezone('US/Mountain')

    # Specified the launch time in UTC:
    launch_time = dt.datetime.strptime(launch_time_from_log, '%Y%m%d%H%M%S')
    launch_time_utc = tz_utc.localize(launch_time)

    # print(launch_time_utc)
    # print(launch_time_from_log[:8])
    
    # Load raw data:
    L0_raw_data = pd.read_csv(ST_file_path + '/no_{}.csv'.format(ST_no))
    
    return launch_time_utc, L0_raw_data

In [66]:
def conversion_L0_L1(loaded_ST_file):

    launch_time_utc = loaded_ST_file[0]
    L0_raw_data = loaded_ST_file[1]
    
    tz_utc = pytz.timezone('UTC')

    # Convert the data time to datetime object:
    L0_raw_data['Time'] = pd.to_datetime(L0_raw_data['Time'], utc=tz_utc)

    # Calculate dew-point temperature in raw data:
    L0_raw_data['dT(degC)'] = dewpoint_from_relative_humidity((L0_raw_data['Temperature(degree C)'].to_numpy() * units.degC).to(units.K), L0_raw_data['Humidity(%)'].to_numpy() / 100.)

    # Convert wind speed in raw data:
    # L0_raw_data['WS(kts)'] = (L0_raw_data['Speed(km/hr)'].to_numpy() * units.kilometer_per_hour).to(units.knot)
    L0_raw_data['WS(m/s)'] = (L0_raw_data['Speed(km/hr)'].to_numpy() * units.kilometer_per_hour).to(units.meter_per_second)

    # Convert wind direction in raw data:
    L0_raw_data.loc[L0_raw_data['Direction(degree)'] <= 180, 'WDIR'] = L0_raw_data['Direction(degree)'] + 180
    L0_raw_data.loc[L0_raw_data['Direction(degree)'] > 180, 'WDIR'] = L0_raw_data['Direction(degree)'] - 180
    L0_raw_data.loc[L0_raw_data['Speed(km/hr)'] == 0, 'WDIR'] = 0
    
    # Find the index of launch time and convert L0 to L1 data:
    L1_data = L0_raw_data[L0_raw_data['Time'] >= launch_time_utc]
    
    # Set Time(sec) in L1 data:
    L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
    # L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).astype('timedelta64[s]')
    
    return L1_data

In [27]:
def output_L1(loaded_ST_file, L1_data, L1_csv_filename):

    # Output L1 data (csv format):

    with open(L1_csv_filename, 'w') as file:

        # Required fields:
        file.write('FileFormat,CSV\n')
        file.write('Year,{}\n'.format(loaded_ST_file[0].year))
        file.write('Month,{:02d}\n'.format(loaded_ST_file[0].month))
        file.write('Day,{:02d}\n'.format(loaded_ST_file[0].day))
        file.write('Hour,{:02d}\n'.format(loaded_ST_file[0].hour))
        file.write('Minute,{:02d}\n'.format(loaded_ST_file[0].minute))
        file.write('Second,{:02d}\n'.format(loaded_ST_file[0].second))

        file.write('Ascending,"true"\n')

        # Optional fields:
        file.write('latitude,40.590000,"units=deg"\n')
        file.write('longitude,-105.141500,"units=deg"\n')
        file.write('altitude,1571.9,"units=m"\n')
        file.write('gpsaltitude,1571.9,"units=m"\n')
        file.write('project,"PRE-CIP-2021"\n')
        file.write('agency,"CSU"\n')
        file.write('sondeid,"{}"\n'.format(ST_no))
        file.write('sondetype,"Storm Tracker"\n')
        file.write('launchsite,"Christman Field"\n')

        # Data headers:
        file.write('Fields,Time,Pressure,Temperature,RH,Speed,Direction,Latitude,Longitude,Altitude,gpsalt\n')
        file.write('Units,sec,mb,deg C,%,m/s,deg,deg,deg,m,m\n')

        # Data fields:

        for index, row in L1_data.iterrows():

            file.write('Data,%6.1f,%7.2f,%5.2f,%5.2f,%6.2f,%6.2f,%9.5f,%9.5f,,%7.1f\n'\
                       % (row['Time(sec)']\
                        , row['Pressure(hPa)']\
                        , row['Temperature(degree C)']\
                        , row['Humidity(%)']\
                        , row['WS(m/s)']\
                        , row['Direction(degree)']\
                        , row['Lat']\
                        , row['Lon']\
                        , row['Height(m)']\
                         )\
                      )

In [67]:
# %%
# MAIN PROGRAM:
t_start = time.process_time()

ST_info = read_in_ST_info(ST_info_file)

for index, row in ST_info.iterrows():

    if dt.datetime.strptime(str(row['Date']), '%Y%m%d') <= final_date:
        
        log_launchT = str(row['Date']) + str(row['Launch_T'])[:6]
        
        st_file_date_path = ST_L0_file_path + '/{}'.format(str(row['Date']))

        loaded_ST_file = load_st_file(str(row['ST_No'])[:4], log_launchT, st_file_date_path)
        
        L1_data = conversion_L0_L1(loaded_ST_file)
        
        L1_output_name = output_path + '/precip21_{}.ST_{}.csv'.format(str(row['Date']) + str(row['Nominal_T']), str(row['ST_No'])[:4])
        output_L1(loaded_ST_file, L1_data, L1_output_name)
        
        print(L1_output_name)

t_finish = time.process_time()
print('Process Time: %f Sec' % (t_finish - t_start))

<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050318.ST_2968.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050418.ST_2134.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050518.ST_2133.csv


/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050818.ST_1841.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021051618.ST_2130.csv


/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

...

/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/

/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021060518.ST_2132.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021060618.ST_2136.csv


/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/

/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021062018.ST_1842.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021062418.ST_1840.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021062518.ST_1838.csv
Process Time: 0.353421 Sec


/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
<ipython-input-66-eeb81d20a1fc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
